In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지에 들어가는 한글을 제대로 보기 위해 한글 폰트 적용
import platform
font_dict = {
    'Linux': 'Noto Sans CJK KR',
    'Darwin': 'Apple SD Gothic Neo', # macOS
    'Windows': 'Malgun Gothic' # MS-Windows
}
try:
    mpl.rc('font', family=font_dict[platform.system()])
except:
    pass
mpl.rc('axes', unicode_minus=False)

%matplotlib inline

# [All about 따릉이 EDA, 4편] 따릉이, 유저 분석해보기 by 흠시

**출처:** https://dailyheumsi.tistory.com/104

**데이터:** 서울특별시 공공자전거 이용정보(시간대별) @[서울 열린데이터 광장](https://data.seoul.go.kr)
  - 대여일시, 대여시간, 대여소번호, 대여소명, 정기권유무, 성별, 연령대, 탄소량, 이동거리, 이동시간 정보를 제공
  - https://data.seoul.go.kr/dataList/OA-15245/F/1/datasetView.do
  - `서울특별시 공공자전거 시간대별 대여정보_201812~201905(1).csv` (용량 103.2MB, 수정일 2019.11.06)
  - `서울특별시 공공자전거 시간대별 대여정보_201812~201905(2).csv` (용량 103.5MB, 수정일 2019.11.06)
  - `서울특별시 공공자전거 시간대별 대여정보_201812~201905(3).csv` (용량 103.6MB, 수정일 2019.11.06)
  - `서울특별시 공공자전거 시간대별 대여정보_201812~201905(4).csv` (용량 44.4MB, 수정일 2019.11.06)
  - `서울특별시 공공자전거 시간대별 대여정보_201812~201905(5).csv` (용량 44.4MB, 수정일 2019.11.06)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(1).csv` (용량 103.6MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(2).csv` (용량 103.6MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(3).csv` (용량 103.5MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(4).csv` (용량 103.5MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(5).csv` (용량 103.5MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(6).csv` (용량 103.5MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(7).csv` (용량 103.6MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(8).csv` (용량 103.5MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(9).csv` (용량 103.3MB, 수정일 2019.12.23)
  - `서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(10).csv` (용량 53.7MB, 수정일 2019.12.23)

> [흠시] 따릉이를 주로 타는 사람들은 과연 누구일까??  
남자일까? 여자일까?  
또, 20대가 주로 탈까? 50대 이상은 잘 타지 않을까?  
한편, 따릉이로 장거리 주행하는 사람들은 대체 누굴까??

> [흠시] 이번 글에서는, 이러한 호기심을 바탕으로, 따릉이 이용자에 대한 분석을 해본다.  
데이터는 <u>2017년 1월 ~ 2018년 12월 따릉이 데이터</u>를 사용한다.

In [ ]:
# 데이터 로드: 원본 데이터는 크기가 너무 크므로 압축된 데이터를 불러온다.
#   서울특별시 공공자전거 시간대별 대여정보_201812~201905(1).csv.gz (23.5MB) ...
#   서울특별시 공공자전거 시간대별 대여정보_201812~201905(5).csv.gz (10.6MB)
#   서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(1).csv.gz (24.9MB) ...
#   서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(10).csv.gz (12.5MB)

from pathlib import Path

데이터_폴더 = Path('../data')
공공자전거_이용정보 = 데이터_폴더 / '서울특별시 공공자전거 이용정보(시간대별)'

files = list(공공자전거_이용정보.glob('*.csv.gz'))
files

In [ ]:
# 먼저 하나의 샘플 데이터 파일을 골라 필요한 컬럼 정보를 살펴보자.

sample = 공공자전거_이용정보 / "서울특별시 공공자전거 이용정보(시간대별)_20190601_20191130(10).csv.gz"
#sample = 공공자전거_이용정보 / "서울특별시 공공자전거 시간대별 대여정보_201812~201905(5).csv.gz"
df = pd.read_csv(sample,
    encoding='cp949'
)
df.head()

In [ ]:
# 운동량과 탄소량 컬럼이 숫자(float64 또는 int64)가 아님에 주의하자.

df.info()

In [ ]:
# 운동량 컬럼을 숫자로 바꾸려면 에러가 발생하므로, 숫자가 아닌 값은 NaN(Not a Number)으로 처리한다.
#   df['운동량'].astype('float64')

df['운동량'].replace({'\\N': np.nan}).astype('float64')

In [ ]:
# 탄소량 컬럼도 숫자로 바꾸려면 에러가 발생하므로, 숫자가 아닌 값은 NaN(Not a Number)으로 처리한다.
#   df['탄소량'].astype('float64')

df['탄소량'].replace({'\\N': np.nan}).astype('float64')

In [ ]:
# 컬럼의 데이터 형식을 미리 정해주자. 이번에는 category 데이터 형식을 사용한다.

column_types = {
    '대여시간': 'int64',
    '대여소번호': 'int64',
    '대여소명': 'category',
    '대여구분코드': 'category',
    '성별': 'category',
    '연령대코드': 'category',
    '이용건수': 'int64',
    '운동량': 'float64',
    '탄소량': 'float64',
    '이동거리': 'int64',
    '사용시간': 'int64',
}

df = pd.read_csv(sample,
    encoding='cp949',
    dtype=column_types,
    na_values='\\N',
    parse_dates=['대여일자']
)
df.info()

In [ ]:
df.tail()

In [ ]:
# 대여소명 카테고리 전체 집합 확인

df['대여소명'].cat.categories

In [ ]:
# 대여구분코드 카테고리 전체 집합 확인

df['대여구분코드'].cat.categories

In [ ]:
# 성별 카테고리 전체 집합 확인

df['성별'].cat.categories

In [ ]:
# 성별은 전부 대문자로 처리한다.

df['성별'] = df['성별'].str.upper().astype('category')
df['성별'].cat.categories

In [ ]:
# 연령대코드 카테고리 전체 집합 확인

df['연령대코드'].cat.categories

In [ ]:
%%time

# 시간대별 대여정보 전체 파일을 읽어들인 후 하나로 합하자. 시간이 꽤 걸릴 수 있다.

df_list = []
for fname in files:
    _df = pd.read_csv(fname,
        encoding='cp949',
        dtype=column_types,
        na_values='\\N',
        parse_dates=['대여일자']
    )
    _df['성별'] = _df['성별'].str.upper().astype('category')
    df_list.append(_df)

df = pd.concat(df_list)

In [ ]:
# 대여일자로 부터 연, 월, 일, 요일 컬럼을 따로 만들어 두면 편리하다.

df['년'] = df['대여일자'].dt.year
df['월'] = df['대여일자'].dt.month
df['일'] = df['대여일자'].dt.day
df['요일'] = df['대여일자'].dt.dayofweek

df.info(memory_usage='deep')

In [ ]:
# 1편에서 사용한 공공자전거 대여소 정보를 불러오자.
# 여기에서는 대여소_구 및 대여소ID 컬럼만 사용한다.

공공자전거_대여소_정보 = 데이터_폴더 / '서울특별시 공공자전거 대여소 정보(19.12.9).xlsx'

rental = pd.read_excel(공공자전거_대여소_정보,
    usecols=['대여소_구', '대여소ID'],
    dtype={'대여소_구': 'category', '대여소ID': 'int64'},
    skipfooter=1
).rename(columns={'대여소ID': '대여소번호'})
rental.info()

In [ ]:
# 시간대별 대여정보와 대여소 정보를 합하자.

df = df.merge(rental, on='대여소번호')
df.head()

In [ ]:
%%time

# 이렇게 데이터 프레임을 매번 만들면 시간이 걸리므로 한번 만들어 놓은 데이터 프레임은 저장하자.
# 이때 확장자를 'pkl.gz'로 만들면 자동으로 압축해 준다.
# 시간이 오래 (몇 분) 걸릴 수 있으므로 주의하자.

# df.to_pickle(데이터_폴더 / '서울특별시 공공자전거 이용정보(시간대별).pkl.gz')

---

In [ ]:
%%time

# 이미 만들어 놓은 데이터 프레임을 불러오자.
# 불러오는 시간은 저장에 비해 매우 짧다.

# df = pd.read_pickle(데이터_폴더 / '서울특별시 공공자전거 이용정보(시간대별).pkl.gz')
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

---
## 1. 성별로 나누어 살펴보기

### 1.1. 남자와 여자, 둘 중에 누가 더 많이 사용할까?

> [흠시] 2년 동안의 남녀 이용량과 이용비율을 시각화 해서 보자.

In [ ]:
# 남녀 이용량 (Series)

use_per_sex = df.groupby('성별')['이용건수'].sum()
use_per_sex

In [ ]:
sex_colors = ['crimson', 'royalblue'] # 인덱스 순서대로 컬러 적용

_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))

use_per_sex.plot.bar(
    rot=0,
    color=sex_colors,
    title="남녀 이용량(2018-2019)",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.ticklabel_format(axis='y', style='plain')
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{height / 1e6:.1f} m", (x+width/2, height), ha='center')

(use_per_sex / use_per_sex.sum()).plot.pie(
    colors=sex_colors, # not color but colors
    autopct="%.1f%%",
    title="남녀 이용비율(2018-2019)",
    ax=ax2
)
ax2.set_frame_on(False)

> [흠시] 너무나 직관적으로 다음과 같이 말할 수 있다.  
> > 남자가 더 많이 사용했고, 남성 이용자가 전체의 62.4% 다.

In [ ]:
print(
    "남자가 여자보다 "
    f"{((use_per_sex['M'] - use_per_sex['F']) / use_per_sex['F']) * 100:.1f}%"
    "(여자 이용기준) 더 많이 이용한다."
)

> [흠시] 한편, 덧붙이면, 남자가 여자보다 65%(여자 이용기준) 더 많이 이용한다.

### 1.2. 년, 월별로 이용자의 남녀 비율 변화가 있었을까?

![](reshaping_pivot.png)
- **출처:** https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#pivot-tables

In [ ]:
# 아래 pivot 명령을 수행하면 에러를 발생한다. 동일한 인덱스(년)와 동일한 컬럼(성별)을 가지는
# 로우(row)가 둘 이상 있기 때문에 어느 것(이용건수)을 값으로 선택할 지 모호하기 때문이다.

# df.pivot(index='년', columns='성별', values='이용건수')

In [ ]:
# 동일한 인덱스와 동일한 컬럼을 가지는 로우들이 여럿 있을 때, 이러한 값들에 대해 sum, mean 등의
# aggregation 함수를 적용할 수 있는 명령이 바로 pivot_table 이다.
# pivot_table의 aggfunc 디폴트 값은 numpy.mean이다.

use_per_year_sex = df.pivot_table(
    index='년', columns='성별', values='이용건수', aggfunc='sum'
)
use_per_year_sex

In [ ]:
# 연도별로 위 값을 백분률로 표시하려면? 먼저 연도별로 총합을 구한 후 인덱스에 따라 나누어주자.

use_per_year_sex = use_per_year_sex.div(
    use_per_year_sex.sum(axis='columns'), axis='index'
)
use_per_year_sex

In [ ]:
# 이번에는 연도별 대신 월별로 성별 이용건수를 구해보자.

use_per_month_sex = df.pivot_table(
    index='월', columns='성별', values='이용건수', aggfunc='sum'
)
use_per_month_sex = use_per_month_sex.div(
    use_per_month_sex.sum(axis='columns'), axis='index'
)
use_per_month_sex

In [ ]:
# 아래에서 범례(legend) 박스의 지정위치(loc)는 bbox_to_anchor (x,y) 또는 (x,y,w,h)로
# 조정할 수 있다. 즉, 그림(ax)의 bbox_to_anchor에 범례 박스의 지정위치가 놓여진다.

sex_colors_dict = {'F': 'crimson', 'M': 'royalblue'} # 컬럼명에 대해 컬러를 지정

_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))                     

use_per_year_sex.plot.barh(
    stacked=True,
    rot=0,
    color=sex_colors_dict,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="년도별 남녀 이용비율",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.legend(frameon=False, loc='center right', bbox_to_anchor=(1, 0.5))

use_per_month_sex.plot.bar(
    stacked=True,
    rot=0,
    color=sex_colors_dict,
    title="월별 남녀 이용비율",
    ax=ax2
)
ax2.set_frame_on(False)
ax2.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5));

> [흠시] 이 역시 다음과 같이 쉽게 말할 수 있을 듯 하다.  
> > 년으로 봤을때는 비율 변화가 없다.  
> > 월로 봤을 때는 타기좋은 날씨(여름 전, 후)에  
> > 여성비율이 최악일 때보다 약 10% 정도 증가한다.

### 1.3. 18년->19년, 유저가 더 많이 증가한 성별은 어딜까?

In [ ]:
# 년도별 성별 이용량(월)은 DataFrame

df_pivot = df.pivot_table(
    index='성별', columns='년', values='이용건수', aggfunc='sum'
)
df_pivot[2019] //= 11 # 2018년은 1개월, 2019년은 11개월
df_pivot

In [ ]:
# 18년 대비 19년도 성별 이용 증가율은 Series

df_pivot_ratio = (df_pivot[2019] - df_pivot[2018]) / df_pivot[2018] * 100
df_pivot_ratio

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))                     

df_pivot.plot.bar(
    rot=0,
    color={2018: 'slateblue', 2019: 'darkslateblue'},
    title="년도별 성별 이용량(월) 변화",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.legend(frameon=False)
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{height / 1e6:.1f} m", (x+width/2, height), ha='center')

df_pivot_ratio.plot.barh(
    rot=0,
    color=sex_colors,
    title="18년 대비 19년도 이용 증가율(%)",
    ax=ax2
)
ax2.set_frame_on(False)
for p in ax2.patches:
    x, y, width, height = p.get_bbox().bounds
    ax2.text(width*1.01, y+height/2, f"{width:.1f}%", va='center')

In [ ]:
print(
    "여자 이용자가 전해대비 "
    f"{((df_pivot.loc['F', 2019] - df_pivot.loc['F', 2018]) / df_pivot.loc['F', 2018]) * 100:.1f}%"
    "로 남자 "
    f"{((df_pivot.loc['M', 2019] - df_pivot.loc['M', 2018]) / df_pivot.loc['M', 2018]) * 100:.1f}%"
    "보다 더 많이 증가했다."
)

> [흠시] 남자 이용자가 전해대비 56.6%로 여자 48.4% 보다 더 많이 증가했다.

### 1.4. 남녀별로, 평균 이용거리, 이용시간 차이가 있을까?

> [흠시] 이번에는 violinplot 을 이용하여 시각화해보자.

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.violinplot(
    x='이동거리',
    y='성별',
    data=df,
    palette=sex_colors_dict,
    ax=ax
)
ax.set_frame_on(False)

In [ ]:
df['이동거리'].max()

> [흠시] 무턱대고 그렸더니, 이상치들까지 같이 표시된다.  
하루 동안 따릉이로 1600000m, 즉 1600키로를 갈 일은 상식적으로 불가능하므로, 극단치가 아니라 이상치라고 봐야한다.

> [흠시] z-score 를 이용하여 이상치 값들을 제거 한 뒤, 다시 그려보면, (z-score 가 3 미만; 99.7%)

In [ ]:
from scipy import stats

df_robust = df[np.abs(stats.zscore(df['이동거리'])) < 3]
df_robust['이동거리'].max()

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.violinplot(
    x='이동거리',
    y='성별',
    data=df_robust,
    palette=sex_colors_dict,
    ax=ax
)
ax.set_frame_on(False)

> [흠시] 이용거리 분포. 이용시간은 이용거리와 선형관계이므로, 굳이 따로 플롯을 그리지 않았다.  
사실, 그래도 분포는 비슷하고, 실질적인 차이를 보려면 중간값을 직접 봐야할듯 하다.

In [ ]:
# 성별로 이동거리와 사용시간의 중간값을 뽑아보자.

df.groupby('성별')[['이동거리', '사용시간']].median()

> [흠시] 중간값을 뽑아보면 다음과 같은 수치가 나온다.  
> > 여자 : 이용거리 2690m, 이용시간 22분  
> > 남자 : 이용거리 2370m, 이용시간 17분

> [흠시] 종합적으로 다음과 같은 결론을 낼 수 있다.  
> > 이용자는 남자가 더 많지만, 일반적으로 한 번 이용시간은 여자보다 짧다.

---
## 2. 연령대로 나누어 살펴보기

### 2.1. 어느 연령대가 제일 많이 사용할까?

> [흠시] 데이터에는 10 단위 기준으로 연령대를 구분해놓았는데, 이를 바로 시각화해서 보자.

In [ ]:
# df['연령대코드'].cat.categories
# Index(['20대', '30대', '40대', '50대', '60대', '70대~', '~10대'], dtype='object')

age_order = ['~10대', '20대', '30대', '40대', '50대', '60대', '70대~']

use_per_age = df.groupby('연령대코드')['이용건수'].sum()
use_per_age = use_per_age.reindex(age_order)
use_per_age

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))                     

use_per_age.plot.bar(
    rot=0,
    color=[f"C{i}" for i in range(len(use_per_age))],
    ylabel="",
    title="연령대 이용량(2018-2019)",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.ticklabel_format(axis='y', style='plain')
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{height / 1e6:.1f} m", (x+width/2, height), ha='center')
    
(use_per_age / use_per_age.sum()).plot.pie(
#    colors=[f"C{i}" for i in range(len(use_per_age))],
    autopct="%.1f%%",
    title="연령대 이용비율(2018-2019)",
    ax=ax2
)
ax2.set_frame_on(False)

> [흠시] 20대 > 30대 > 40대 > 50대 순으로 이용자가 많다.  
20대 이용자는 전체의 48.2%로, 이용 비율은 50대까지 선형적으로 감소한다.

### 2.2. 년, 월별로 연령대 비율 변화가 있었을까?

> [흠시] 연령대 역시 성별과 마찬가지로 년, 월로 변화 비율을 봐보자.

In [ ]:
# 연별 연령대 비율 변화

use_per_year_age = df.pivot_table(
    index='년', columns='연령대코드', values='이용건수', aggfunc='sum'
)
use_per_year_age = use_per_year_age.div(
    use_per_year_age.sum(axis='columns'), axis='index'
)
use_per_year_age = use_per_year_age[age_order]
use_per_year_age

In [ ]:
# 월별 연령대 비율 변화

use_per_month_age = df.pivot_table(
    index='월', columns='연령대코드', values='이용건수', aggfunc='sum'
)
use_per_month_age = use_per_month_age.div(
    use_per_month_age.sum(axis='columns'), axis='index'
)
use_per_month_age = use_per_month_age[age_order]
use_per_month_age

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))                     

use_per_year_age.plot.barh(
    stacked=True,
    rot=0,
    xlabel="",
    title="년도별 연령대 이용비율(%)",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.legend(frameon=False, loc='upper center', ncol=7)
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{width * 100:.1f}", (x+width/2, y+height/2), ha='center', va='center')

use_per_month_age.plot.bar(
    stacked=True,
    rot=0,
    title="월별 연령대 이용비율(%)",
    ax=ax2
)
ax2.set_frame_on(False)
ax2.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
for p in ax2.patches:
    x, y, width, height = p.get_bbox().bounds
    ax2.annotate(f"{height * 100:.1f}", (x+width/2, y+height/2), ha='center', va='center')

> [흠시] 먼저, 왼쪽 플롯을 보면, 2017년에서 2018년으로 갈 때, 연령대 비율 변화가 거의 없었음을 알 수 있다.  
있다면, 50대가 6.7 -> 5.8% 로 줄고, 10대가 1.9 -> 2.6% 로 증가한 정도가 있겠다.

> [흠시] 오른쪽 플롯은 월별로 연령대별 이용자 비율을 나타내는데, 각 연령대마다 월이 지남에 따라 어떤 경향성을 가지는지 알기가 힘들다.  
질문을 좀 바꿔, 어떤 연령대끼리 이용비율이 변하는데 있어 비슷한 경향을 가지는지 조사해보자.  
피어슨 상관계수를 이용하여, 클러스터 맵으로 시각화하면 쉽게 알아낼 수 있다.

In [ ]:
sns.clustermap(
    use_per_month_age.corr(),
    cmap='coolwarm',
    vmin=-1, vmax=1,
    annot=True,
    fmt=".1f"
);

> [흠시] 30, 40, 50, 60대가 월별 이용비율 변화에 비슷한 경향을 지니고 있다.  
한편, 20대와 30대는 상관계수가 0.9로, 굉장히 강한 음의 선형관계를 가지는 것도 알 수 있다.  
정리하면 다음과 같다.  
> > 년으로 봤을때는 비율 변화가 없다.  
> > 월로 봤을 때는 30~60대가 비슷한 경향으로 변화하고,  
> > 특히, 20대와 30대는 정반대 변화 경향을 갖는다.

### 2.3. 18년->19년, 어느 연령대 유저가 더 많이 증가했을까?

In [ ]:
# 연령대별 연간 이용건수(월) (DataFrame)

df_pivot = df.pivot_table(
    index=['연령대코드'], columns='년', values='이용건수', aggfunc='sum'
)
df_pivot[2019] //= 11
df_pivot = df_pivot.reindex(age_order)
df_pivot

In [ ]:
# 연령대별 18년 대비 19년도 이용 증가율 (Series)

df_pivot_ratio = (df_pivot[2019] - df_pivot[2018]) / df_pivot[2018] * 100
df_pivot_ratio.sort_values(inplace=True)
df_pivot_ratio

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))                     

df_pivot.plot.bar(
    rot=0,
    color={2018: 'slateblue', 2019: 'darkslateblue'},
    title="년도별 연령대 이용량(월) 변화",
    ax=ax1
)
ax1.set_frame_on(False)
ax1.legend(frameon=False)

df_pivot_ratio.plot.barh(
    rot=0,
    color=[f"C{i}" for i in range(len(df_pivot_ratio))],
    xlabel="",
    title="18년 대비 19년도 이용 증가율(%)",
    ax=ax2
)
ax2.set_frame_on(False)

> [흠시] 전반적으로 모든 연령대가 다 증가하였고,  
특히, 50대 유저가 전해대비 약 80% 증가로 가장 많이 증가했다.

### 2.4. 연령별로, 평균 이용거리, 이용시간 차이가 있을까?

> [흠시] violinplot 보다 boxplot 이 데이터를 더 잘 보여줄듯 하여, 이번에는 boxplot 으로 시각화 해본다.

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.boxplot(
    x='이동거리',
    y='연령대코드',
    data=df,
    ax=ax
)
ax.set_frame_on(False)

> [흠시] 그대로 그리면, 이상치가 너무 많아서, z-score < 3 으로 ~몇번 거른뒤~ 시각화 하자.

In [ ]:
from scipy import stats

df_robust = df[np.abs(stats.zscore(df['이동거리'])) < 1]
df_robust['이동거리'].max()

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))
sns.boxplot(
    x='이동거리',
    y='연령대코드',
    data=df_robust,
    ax=ax
)
ax.set_frame_on(False)

> [흠시] 뭐, 상식적이게도, 역시 젊을수록 오래 이동한다. 10대를 제외하고는 말이다.  
그래도 중간값은 생각보다 그렇게 큰 차이를 보이지는 않는다.  
> > 역시, 젊은 사람들(10대 제외)이 일반적으로 오래탄다.

---
## 3. 연령 x 성별로 좀 더 살펴보기

> [흠시] 이번에는 각 연령대별을 이루고 있는 성별,  
혹은 각 성별 내 연령대비율에 대해서 좀 더 자세히 살펴본다.


### 3.1. 연령대 내 성별 비율

In [ ]:
df_pivot = df.pivot_table(
    index="연령대코드", columns="성별", values="이용건수", aggfunc="sum"
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot.sort_values('F', inplace=True)
df_pivot

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    color=sex_colors_dict,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="연령대별 성별 비율(%)",
    figsize=(10, 5)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
for p in ax.patches:
    x, y, width, height = p.get_bbox().bounds
    ax.annotate(f"{width * 100:.1f}", (x+width/2, y+height/2), ha='center', va='center')

> [흠시] 먼저 전반적으로 전 연령대에서 남성비율이 많기는 하지만, 여성비율이 26~46%로 차이가 난다.  
먼저 70대 이상에서, 여성 비율이 46.2%로 제일 많고,  
주 이용자였던 20대 -> 40대로 갈수록 여성비율이 감소함을 알 수 있다.  
> > 전 연령대에서 남성비율이 53~73%로 여성보다 많지만,  
> > 주 이용자 연령인 20대는 남성이 55.6%로 여성과 10% 차이밖에 안난다.

### 3.2. 성별 내 연령대 비율

In [ ]:
df_pivot = df.pivot_table(
    index="성별", columns="연령대코드", values="이용건수", aggfunc="sum"
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot = df_pivot.reindex(columns=age_order)
df_pivot

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="성별별 연령 비율(%)",
    figsize=(10, 5)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
for p in ax.patches:
    x, y, width, height = p.get_bbox().bounds
    ax.annotate(f"{width * 100:.1f}", (x+width/2, y+height/2), ha='center', va='center')

> [흠시] 여자의 경우 남자보다 20대 비율이 더 높음을 알 수 있다.  
반대로, 남자의 경우 20대 비율은 여자보다 적지만, 나머지 30대와 40대 비율이 조금 더 높은 것을 알 수 있다.  
> > 여자의 경우, 20대에 이용자 비율이 남자보다 14% 정도 더 몰려있다.

---
## 4. 지역별로 나누어 살펴보기

### 4.1. 지역별로 연령이나 성별 비율이 다르진 않을까?

In [ ]:
df_pivot = df.pivot_table(
    index='대여소_구', columns='연령대코드', values='이용건수', aggfunc='sum'
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot.sort_values('20대', inplace=True)
df_pivot

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="지역별 이용자 연령대 비율",
    figsize=(10, 8)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5));

> [흠시] 위 플롯이 한 눈에 들어오지 않는다면, 주 이용층인 그냥 20대만 집중해서 살펴보자.  
먼저, 광진구나 노원구의 경우, 이용자의 약 60%가 20대임을 알 수 있다.  
한편, 서초, 금천의 경우 40%가 안된다.  
20대의 비중이 크게는 지역별로 20%나 차이가 난다는 것이다.

> [흠시] 그렇다면, 특정 지역구 내, 대여소에도 이런 차이가 있을까?  
20대의 비중이 높은 광진구 내 대여소들만 살펴보도록 하자.

In [ ]:
gwangjin = df[df['대여소_구'] == '광진구']

df_pivot = gwangjin.pivot_table(
    index="대여소명", columns='연령대코드', values='이용건수', aggfunc='sum'
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot.sort_values('20대', inplace=True)
df_pivot

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="광진구 대여소별 이용자 연령대 비율",
    figsize=(10, 12)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5));

> [흠시] 대여소 역시, 20대 비중이 많게는 80%를 넘고, 적게는 30%도 안된다.  
20대 비중이 많은 상위 대여소들을 살펴보면, 세종대학교, 건국대학교 등.. 대부분 대학 근처 대여소들이다.  
한편, 20대 비중이 낮은 곳은, 아파트, 중학교, 도서관 등.. 상식적으로 20대의 이용비율이 낮을만한 곳들이다.

> [흠시] 그렇다면, 이번엔 지역별로 성별 비율을 보면 어떨까?

In [ ]:
df_pivot = df.pivot_table(
    index='대여소_구', columns='성별', values='이용건수', aggfunc='sum'
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot.sort_values('M', inplace=True)
df_pivot = df_pivot.reindex(columns=['M', 'F'])
df_pivot

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    color=sex_colors_dict,
    xlabel="", # barh는 xlabel과 ylabel의 위치가 다름
    title="지역별 이용자 성별 비율",
    figsize=(10, 8)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5));

> [흠시] 뭐 역시나, 전 지역에서 남성 이용자가 더 많은 패턴은 동일하다.  
지역별 남녀 차이도 10% 내외로 그렇게 두드러지지 않는다.

> [흠시] 정리하면,  
> > 지역별, 그리고 지역 내 대여소마다 이용자의 연령비율은 확연히 다르다.  
> > 성별비율은 그다지 유의미하게 다르지 않다.

---
## 5. 다루지 않은 내용들

사실 데이터를 열어보면, 더 확인하고 싶었고, 확인할 수 있는 내용들이 꽤 있었다.  
예를 들어, 장거리 주행자는 누구일까? 라든가, 따릉이 속력 분포는 어떻게 되어있을까? 등 이라든지...

하지만 여기서 다루지 않는 이유는, 아니 사실 다뤄봤음에도 글을 쓰지 않는 이유는  
1.  장거리 주행자들? 위에서 살펴본 연령x성별 분포와 유사하다. 따로 살펴보는게 별 의미가 없을 듯 싶어서.
2.  이용거리와 이용시간에 이상값들이 너무 많다. 예를 들어, 이용시간이 0이라든가, 이용거리가 300000이라든가..  
    문제는, 이상치인지, 정상적인 범주에서의 극단값인지 명확하지가 않다.  
    따라서, 구태여 위험을 무릎쓴 해석은 다른 분께 넘겨본다.이상치 많아서 의미없음.